In [2]:
import pickle
from distributed import Client, LocalCluster
from dask_jobqueue import SGECluster
import numpy as np

In [3]:
def experiment(dataset, Loader, methods):
    datasets = [dataset]
    corruptions = [i / 10 for i in range(10)]
    nb_folds = 10
    floader = FoldsLoader(Loader, cross_on_weak_loss=False)
    C_sigmas = np.logspace(-2, 2, 5)
    C_lambdas = np.logspace(-3, 3, 7)
    np.random.seed(0)
    res_dict = iterate_on_datasets(datasets, corruptions, nb_folds, floader,
                                   methods, C_sigmas, C_lambdas, low_rank=200)
    if floader.wk:
        dataset += '_weak'
    with open(SAVE_DIR / (dataset + '.pk'), 'wb') as f:
        pickle.dump(res_dict, f)
    return res_dict


def MULAN_experiment(dataset):
    methods = ['plasp', 'low rank', 'diffusion']
    return experiment(dataset, MULANLoader, methods)


def LIBSVM_experiment(dataset):
    methods = ['plasp', 'averaging', 'diffusion']
    return experiment(dataset, LIBSVMLoader, methods)


def synthetic_experiment(dataset):
    methods = ['plasp', 'diffrac', 'diffusion', 'low rank']
    return experiment(dataset, Synthesizer, methods)

In [26]:
def get_dask_cluster(on_server, n_workers):
    if on_server:
        env_extra = [
            'conda activate main',
            'source /home/vcabanne/.bashrc',
            'export LANG=en_US.UTF-8',
            'export LC_ALL=en_US.UTF-8'
        ]
        log_directory = "/sequoia/data2/vcabanne/logs"
        local_directory = '/sequoia/data2/vcabanne/dask-spill'

        cluster = SGECluster(queue='all.q,bigmem.q',
                             resource_spec='h_vmem=12000M,mem_req=8000M',
                             walltime='720:00:00',
                             name='plasp',
                             cores=1,
                             memory='8000m',
                             processes=1,
                             interface='ib0',
                             env_extra=env_extra,
                             log_directory=log_directory,
                             local_directory=local_directory)
        cluster.scale(n_workers)

    else:
        cluster = LocalCluster(n_workers=n_workers, threads_per_worker=1,
                               processes=False)

    return cluster


In [27]:
# architecture
on_server = True
n_workers = 1
cluster = get_dask_cluster(on_server, n_workers)
client = Client(cluster)

# MULAN data
MULAN_datasets = MULANLoader.datasets
MULAN_datasets = [
    'birds',
    'CAL500',
    'emotions',
    'enron',
    'flags',
    'genbase',
    'medical',
    'scene',
    'yeast'
]
futures = client.map(MULAN_experiment, MULAN_datasets)
# results = client.gather(futures)

# LIBSSM data
LIBSVM_datasets = LIBSVMLoader.datasets
LIBSVM_datasets = [
    'iris',
    'wine',
    'svmguide2',
    'glass',
    'svmguide4',
    'vehicle',
    'vowel',
    'dna'
]
futures = client.map(LIBSVM_experiment, LIBSVM_datasets)
# results = client.gather(futures)

# Synthetic data
datasets = ['CL-1000', 'CL-400', 'CL-100',
            'ML-1000', 'ML-400', 'ML-100']
futures = client.map(synthetic_experiment, datasets)
results = client.gather(futures)

/sequoia/data1/vcabanne/miniconda3/envs/main/lib/python3.7/site-packages/distributed/deploy/local.py:138: UserWarning: diagnostics_port has been deprecated. Please use `dashboard_address=` instead
  "diagnostics_port has been deprecated. "
/sequoia/data1/vcabanne/miniconda3/envs/main/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


NameError: name 'MULANLoader' is not defined

In [28]:
cluster

In [30]:
client

Client Scheduler: tcp://10.149.0.19:41047 Dashboard: http://10.149.0.19:47144/status,Cluster Workers: 1 Cores: 1 Memory: 8.00 GB


In [17]:
client.

AttributeError: 'Client' object has no attribute 'get_logs'

In [20]:
cluster.scheduler.processing

<class 'distributed.scheduler._StateLegacyMapping'>({})